In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Read the gold analytics table
gold_df = spark.table("stock_market.gold_stock_analytics")

print(f"Total records: {gold_df.count()}")
print(f"Stocks: {gold_df.select('symbol').distinct().count()}")
print(f"Date range: {gold_df.agg(min('date'), max('date')).collect()[0]}")

# Show latest data for all stocks
gold_df.filter(col("date") == gold_df.agg(max("date")).collect()[0][0]) \
    .select("symbol", "close", "daily_change_pct", "trend_signal") \
    .orderBy(col("daily_change_pct").desc()) \
    .show()

In [0]:
# Calculate performance over different time periods
from pyspark.sql.window import Window
from datetime import timedelta

# Get the latest date
latest_date = gold_df.agg(max("date")).collect()[0][0]

# Calculate returns over different periods
def calculate_returns(df, days_back, period_name):
    # Get data for the last N days
    date_threshold = latest_date - timedelta(days=days_back)
    
    period_df = df.filter(col("date") >= date_threshold)
    
    # Get first and last prices for each symbol
    return period_df.groupBy("symbol") \
        .agg(
            first("close", ignorenulls=True).alias(f"{period_name}_start_price"),
            last("close", ignorenulls=True).alias(f"{period_name}_end_price")
        ) \
        .withColumn(f"{period_name}_return_pct", 
            round((col(f"{period_name}_end_price") - col(f"{period_name}_start_price")) / col(f"{period_name}_start_price") * 100, 2))

# 7-day and 30-day performance
weekly_perf = calculate_returns(gold_df, 7, "weekly")
monthly_perf = calculate_returns(gold_df, 30, "monthly")

# Combine
portfolio_perf = weekly_perf.join(monthly_perf, "symbol")

portfolio_perf.select("symbol", "weekly_return_pct", "monthly_return_pct") \
    .orderBy(col("monthly_return_pct").desc()) \
    .show()

In [0]:
# Calculate volatility and risk metrics for each stock
risk_metrics = gold_df.groupBy("symbol").agg(
    round(avg("volatility_7_day"), 2).alias("avg_volatility"),
    round(stddev("daily_change_pct"), 2).alias("daily_volatility"),
    round(max("daily_change_pct"), 2).alias("max_daily_gain"),
    round(min("daily_change_pct"), 2).alias("max_daily_loss")
)

# Join with performance
portfolio_summary = portfolio_perf \
    .join(risk_metrics, "symbol") \
    .withColumn("risk_adjusted_return", 
        round(col("monthly_return_pct") / col("avg_volatility"), 2))

# Show complete portfolio summary
portfolio_summary.select(
    "symbol", 
    "monthly_return_pct", 
    "avg_volatility",
    "risk_adjusted_return",
    "max_daily_gain",
    "max_daily_loss"
).orderBy(col("risk_adjusted_return").desc()).show()

In [0]:
# Write portfolio summary to new gold table
portfolio_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("stock_market.gold_portfolio_summary")

print("✓ Portfolio summary table created")

# Also create a daily portfolio comparison table
daily_comparison = gold_df.filter(col("date") >= date_sub(current_date(), 30)) \
    .select(
        "date", 
        "symbol", 
        "close", 
        "daily_change_pct", 
        "ma_7_day", 
        "volatility_7_day",
        "trend_signal"
    )

daily_comparison.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("stock_market.gold_daily_comparison")

print("✓ Daily comparison table created")

# Show what we created
print(f"\nPortfolio Summary: {spark.table('stock_market.gold_portfolio_summary').count()} stocks")
print(f"Daily Comparison: {spark.table('stock_market.gold_daily_comparison').count()} records")